Kartik Rajesh (I032)

Krish Mehta (I033)

Krish Makhija (I034)

Twitter Sentiment Analysis Using Classification Techniques

In [1]:
#importing the required libraries
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#reading of data set(training data)
tweets_train = pd.read_csv('twitter_training.csv')
tweets_train.columns = ['Tweet ID', 'Users','Sentiment','Content']
tweets_train.head()

,Tweet ID,Users,Sentiment,Content
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [3]:
tweets_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74681 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Tweet ID   74681 non-null  int64 
 1   Users      74681 non-null  object
 2   Sentiment  74681 non-null  object
 3   Content    73995 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [4]:
tweets_train.shape

(74681, 4)

In [5]:
#Reading of data set(test data)
tweets_test = pd.read_csv('twitter_validation.csv')
tweets_test.columns = ['Tweet ID', 'Users','Sentiment','Content']
tweets_test.head()

,Tweet ID,Users,Sentiment,Content
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [6]:
tweets_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Tweet ID   999 non-null    int64 
 1   Users      999 non-null    object
 2   Sentiment  999 non-null    object
 3   Content    999 non-null    object
dtypes: int64(1), object(3)
memory usage: 31.3+ KB


In [7]:
tweets_test.shape

(999, 4)

**PRE-PROCESSING**

In [8]:
#deleting of nan rows from training data
tweets_train.dropna(axis = 0, subset = 'Content', inplace = True)
tweets_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73995 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Tweet ID   73995 non-null  int64 
 1   Users      73995 non-null  object
 2   Sentiment  73995 non-null  object
 3   Content    73995 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.8+ MB


In [9]:
#dropping duplicate values
train_tweets_dup = tweets_train[tweets_train.duplicated(subset = ['Tweet ID', 'Users','Sentiment','Content'],keep = False)]
train_tweets_dup

,Tweet ID,Users,Sentiment,Content
17,2404,Borderlands,Positive,that was the first borderlands session in a lo...
19,2404,Borderlands,Positive,that was the first borderlands session in a lo...
24,2405,Borderlands,Negative,The biggest disappointment of my life came a y...
25,2405,Borderlands,Negative,The biggest disappointment of my life came a y...
47,2409,Borderlands,Neutral,Blaming Sight for Tardiness! A little bit of b...
...,...,...,...,...
74588,9185,Nvidia,Neutral,Heard people are having issues with ordering t...
74615,9190,Nvidia,Positive,This news about the Nvidia 3000 series is ligi...
74618,9190,Nvidia,Positive,This news about the Nvidia 3000 series is ligi...
74634,9193,Nvidia,Positive,Love EVERYTHING about it.


In [10]:
tweets_train.drop(train_tweets_dup.index, axis = 0, inplace = True)

In [11]:
tweets_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69690 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Tweet ID   69690 non-null  int64 
 1   Users      69690 non-null  object
 2   Sentiment  69690 non-null  object
 3   Content    69690 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.7+ MB


In [12]:
tweets_train.head(20)

,Tweet ID,Users,Sentiment,Content
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
5,2402,Borderlands,Positive,So I spent a few hours making something for fu...
6,2402,Borderlands,Positive,So I spent a couple of hours doing something f...
7,2402,Borderlands,Positive,So I spent a few hours doing something for fun...
8,2402,Borderlands,Positive,So I spent a few hours making something for fu...
9,2402,Borderlands,Positive,2010 So I spent a few hours making something f...


In [13]:
#Exploring on test data set
test_duplicate = tweets_test[tweets_test.duplicated(subset=['Tweet ID', 'Users', 'Sentiment', 'Content'], keep=False)]
test_duplicate

,Tweet ID,Users,Sentiment,Content


In [14]:
#Converting the datatypes for preprocessing
tweets_train['Users'] = tweets_train["Users"].apply(str)
tweets_train["Sentiment"] = tweets_train["Sentiment"].astype("category")
tweets_train["Content"] = tweets_train["Content"].apply(str)
tweets_train.head()

,Tweet ID,Users,Sentiment,Content
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [15]:
tweets_test['Users'] = tweets_test["Users"].apply(str)
tweets_test["Sentiment"] = tweets_test["Sentiment"].astype("category")
tweets_test["Content"] = tweets_test["Content"].apply(str)
tweets_test.head()

,Tweet ID,Users,Sentiment,Content
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
label_encoder = LabelEncoder()
label_encoder.fit(tweets_train["Sentiment"])
tweets_train["Sentiment"]= label_encoder.transform(tweets_train["Sentiment"])
tweets_test["Sentiment"]= label_encoder.transform(tweets_test["Sentiment"])
tweets_train.head(25)

,Tweet ID,Users,Sentiment,Content
0,2401,Borderlands,3,I am coming to the borders and I will kill you...
1,2401,Borderlands,3,im getting on borderlands and i will kill you ...
2,2401,Borderlands,3,im coming on borderlands and i will murder you...
3,2401,Borderlands,3,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,3,im getting into borderlands and i can murder y...
5,2402,Borderlands,3,So I spent a few hours making something for fu...
6,2402,Borderlands,3,So I spent a couple of hours doing something f...
7,2402,Borderlands,3,So I spent a few hours doing something for fun...
8,2402,Borderlands,3,So I spent a few hours making something for fu...
9,2402,Borderlands,3,2010 So I spent a few hours making something f...


In [18]:
tweets_test.head()

,Tweet ID,Users,Sentiment,Content
0,352,Amazon,2,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,1,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,1,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,2,Now the President is slapping Americans in the...
4,6273,FIFA,1,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [19]:
#Changing the training data set to a clean data set
def tweets_cleaning(Content):
    Content = re.sub("@[A-Za-z0-9]+","", Content)
    Content = re.sub("https?://[A-Za-z0-9./]+","" ,Content)
    Content = re.sub("[^ A-Za-z0-9.!?']","", Content)
    Content = re.sub(" +"," ", Content)
    return Content

In [20]:
#clean set for training data
clean_content = [tweets_cleaning(Content) for Content in tweets_train["Content"]]
tweets_train["clean content"] = clean_content
tweets_train.head(140)

,Tweet ID,Users,Sentiment,Content,clean content
0,2401,Borderlands,3,I am coming to the borders and I will kill you...,I am coming to the borders and I will kill you...
1,2401,Borderlands,3,im getting on borderlands and i will kill you ...,im getting on borderlands and i will kill you all
2,2401,Borderlands,3,im coming on borderlands and i will murder you...,im coming on borderlands and i will murder you...
3,2401,Borderlands,3,im getting on borderlands 2 and i will murder ...,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,3,im getting into borderlands and i can murder y...,im getting into borderlands and i can murder y...
...,...,...,...,...,...
146,2425,Borderlands,1,"""What terrible bitch!""",What terrible bitch!
147,2425,Borderlands,1,"""... What a bitch! """,... What a bitch!
148,2425,Borderlands,1,"""What a miracle!""",What a miracle!
149,2426,Borderlands,3,Ok I am LOVING THIS (also playing Bombs Over B...,Ok I am LOVING THIS also playing Bombs Over Ba...


In [21]:
#clean set for test data
clean_content = [tweets_cleaning(Content) for Content in tweets_test["Content"]]
tweets_test["clean content"] = clean_content
tweets_test.head(25)

,Tweet ID,Users,Sentiment,Content,clean content
0,352,Amazon,2,BBC News - Amazon boss Jeff Bezos rejects clai...,BBC News Amazon boss Jeff Bezos rejects claims...
1,8312,Microsoft,1,@Microsoft Why do I pay for WORD when it funct...,Why do I pay for WORD when it functions so po...
2,4371,CS-GO,1,"CSGO matchmaking is so full of closet hacking,...",CSGO matchmaking is so full of closet hacking ...
3,4433,Google,2,Now the President is slapping Americans in the...,Now the President is slapping Americans in the...
4,6273,FIFA,1,Hi @EAHelp I’ve had Madeleine McCann in my cel...,Hi Ive had Madeleine McCann in my cellar for t...
5,7925,MaddenNFL,3,Thank you @EAMaddenNFL!! \n\nNew TE Austin Hoo...,Thank you !! New TE Austin Hooper in the ORANG...
6,11332,TomClancysRainbowSix,3,"Rocket League, Sea of Thieves or Rainbow Six: ...",Rocket League Sea of Thieves or Rainbow Six Si...
7,1107,AssassinsCreed,3,my ass still knee-deep in Assassins Creed Odys...,my ass still kneedeep in Assassins Creed Odyss...
8,2069,CallOfDuty,1,FIX IT JESUS ! Please FIX IT ! What In the wor...,FIX IT JESUS ! Please FIX IT ! What In the wor...
9,3185,Dota2,3,The professional dota 2 scene is fucking explo...,The professional dota 2 scene is fucking explo...


In [22]:
#Deleting extra columns
columns = ['Tweet ID','Users','Content']
tweets_train.drop(columns, axis = 1, inplace = True)
tweets_test.drop(columns, axis = 1, inplace = True)

In [23]:
#Printing the clean data set with the required columns(training dataset)
tweets_train.head()

,Sentiment,clean content
0,3,I am coming to the borders and I will kill you...
1,3,im getting on borderlands and i will kill you all
2,3,im coming on borderlands and i will murder you...
3,3,im getting on borderlands 2 and i will murder ...
4,3,im getting into borderlands and i can murder y...


In [24]:
#Printing the clean data set with the required columns(training dataset)
tweets_test.head()

,Sentiment,clean content
0,2,BBC News Amazon boss Jeff Bezos rejects claims...
1,1,Why do I pay for WORD when it functions so po...
2,1,CSGO matchmaking is so full of closet hacking ...
3,2,Now the President is slapping Americans in the...
4,1,Hi Ive had Madeleine McCann in my cellar for t...


**CLASSIFICATION TECHNIQUES**

**1) Logistic Regression**

In [25]:
#importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, accuracy_score

In [26]:
#splitting data into train and test data
X_train, X_test, y_train, y_test = train_test_split(tweets_train["clean content"], tweets_train["Sentiment"], test_size = 0.2, random_state = 42)

In [27]:
#vectorizing the text data
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [28]:
#LG model
logistic_reg = LogisticRegression(max_iter = 1000)
logistic_reg

LogisticRegression(max_iter=1000)

In [29]:
#fitting the model
logistic_reg.fit(X_train_vec, y_train)

LogisticRegression(max_iter=1000)

In [30]:
#making predictions
lg_pred = logistic_reg.predict(X_test_vec)
lg_pred[:5]

array([0, 3, 0, 3, 3])

In [31]:
#classification report
print("Logistic regression Model: ")
print(classification_report(y_test, lg_pred))

Logistic regression Model: 
              precision    recall  f1-score   support

           0       0.85      0.74      0.79      2446
           1       0.82      0.86      0.84      4105
           2       0.85      0.78      0.81      3502
           3       0.77      0.85      0.81      3885

    accuracy                           0.81     13938
   macro avg       0.82      0.81      0.81     13938
weighted avg       0.82      0.81      0.81     13938



In [32]:
print("Accuracy for the Logistic Regression Model Classification is: ")
print(accuracy_score(y_test, lg_pred))

Accuracy for the Logistic Regression Model Classification is: 
0.8141770698809011


In [33]:
#probabilities of each sentiment
logistic_prob = logistic_reg.predict_proba(X_test_vec)
print("Probabilities of sentiments (Positive, Neutral, Negative):")
for i, (instance, prob) in enumerate(zip(X_test[:10], logistic_prob[:10])):  # Printing probabilities of first 5 instances
    print(f"Instance {i + 1}:")
    print(f"Text: {instance}")
    print(f"Irrelevant: {prob[3]}")
    print(f"Positive: {prob[0]}")
    print(f"Neutral: {prob[2]}")
    print(f"Negative: {prob[1]}")

Probabilities of sentiments (Positive, Neutral, Negative):
Instance 1:
Text: A ban has been imposed on Battlefield 4 player POLYSTATION7 see details bf4db.com player ban 100...
Irrelevant: 0.0002688346374634863
Positive: 0.9980688407378353
Neutral: 0.001313602628683104
Negative: 0.0003487219960181072
Instance 2:
Text:  
Irrelevant: 0.3143428074990371
Positive: 0.16456146715518824
Neutral: 0.21601318306134945
Negative: 0.30508254228442533
Instance 3:
Text: Thanks. Correction emailed
Irrelevant: 0.1391532360250882
Positive: 0.4492334104054828
Neutral: 0.07227652963471069
Negative: 0.33933682393471826
Instance 4:
Text: I love my heart stone.
Irrelevant: 0.5469748333670549
Positive: 0.14302737631916548
Neutral: 0.24205198613872597
Negative: 0.06794580417505369
Instance 5:
Text:  GhostRecon the games have their problems but I've definitely enjoyed my time keeping up the good work. pic.twitter.com rUTLeRbg4P
Irrelevant: 0.900417097638453
Positive: 0.0005235127318849251
Neutral: 0.08472467514

**2) Decision Tree**

In [34]:
#importing of libraries required
from sklearn.tree import DecisionTreeClassifier

In [35]:
#Decision Tree Model
decision_tree_model = DecisionTreeClassifier()
decision_tree_model

DecisionTreeClassifier()

In [36]:
#Fitting the model
decision_tree_model.fit(X_train_vec,y_train)

DecisionTreeClassifier()

In [37]:
#Making predictions
decision_tree_pred = decision_tree_model.predict(X_test_vec)
decision_tree_pred[:5]

array([0, 2, 1, 3, 2])

In [38]:
#classification report
print("Decision Tree Model: ")
print(classification_report(y_test, decision_tree_pred))

Decision Tree Model: 
              precision    recall  f1-score   support

           0       0.74      0.68      0.71      2446
           1       0.76      0.78      0.77      4105
           2       0.74      0.75      0.74      3502
           3       0.76      0.77      0.76      3885

    accuracy                           0.75     13938
   macro avg       0.75      0.75      0.75     13938
weighted avg       0.75      0.75      0.75     13938



In [39]:
#prinint of accuracy
print("The accuracy of the Decision Tree Model Classification is: ")
print(accuracy_score(y_test, decision_tree_pred))

The accuracy of the Decision Tree Model Classification is: 
0.7526904864399483


In [40]:
#probabilities of each sentiment
dt_prob = decision_tree_model.predict_proba(X_test_vec)
print("Probabilities of sentiments (Irrelevant, Positive, Neutral, Negative):")
for i, (instance, prob) in enumerate(zip(X_test[:5], dt_prob[:5])):  # Printing probabilities of first 5 instances
    print(f"Instance {i + 1}:")
    print(f"Text: {instance}")
    print(f"Irrelevant: {prob[3]}")
    print(f"Positive: {prob[0]}")
    print(f"Neutral: {prob[2]}")
    print(f"Negative: {prob[1]}")

Probabilities of sentiments (Irrelevant, Positive, Neutral, Negative):
Instance 1:
Text: A ban has been imposed on Battlefield 4 player POLYSTATION7 see details bf4db.com player ban 100...
Irrelevant: 0.0
Positive: 1.0
Neutral: 0.0
Negative: 0.0
Instance 2:
Text:  
Irrelevant: 0.2356164383561644
Positive: 0.2
Neutral: 0.30684931506849317
Negative: 0.25753424657534246
Instance 3:
Text: Thanks. Correction emailed
Irrelevant: 0.0
Positive: 0.0
Neutral: 0.0
Negative: 1.0
Instance 4:
Text: I love my heart stone.
Irrelevant: 1.0
Positive: 0.0
Neutral: 0.0
Negative: 0.0
Instance 5:
Text:  GhostRecon the games have their problems but I've definitely enjoyed my time keeping up the good work. pic.twitter.com rUTLeRbg4P
Irrelevant: 0.0
Positive: 0.0
Neutral: 1.0
Negative: 0.0


**3) K-Nearest Neighbours(KNN)**

In [41]:
#importing the required libraries
from sklearn.neighbors import KNeighborsClassifier

In [42]:
#knn model
knn_model = KNeighborsClassifier()
knn_model

KNeighborsClassifier()

In [43]:
#fitting the model
knn_model.fit(X_train_vec, y_train)

KNeighborsClassifier()

In [44]:
knn_pred = knn_model.predict(X_test_vec)
knn_pred[:5]

array([0, 1, 3, 2, 2])

In [45]:
#classification report
print("K-Nearest Neighbors(KNN) Model Classification: ")
print(classification_report(y_test, knn_pred))

K-Nearest Neighbors(KNN) Model Classification: 
              precision    recall  f1-score   support

           0       0.82      0.81      0.82      2446
           1       0.82      0.87      0.84      4105
           2       0.85      0.81      0.83      3502
           3       0.84      0.82      0.83      3885

    accuracy                           0.83     13938
   macro avg       0.83      0.83      0.83     13938
weighted avg       0.83      0.83      0.83     13938



In [46]:
#accuracy
print("The accuracy for KNN Model Classification is: ")
print(accuracy_score(y_test, knn_pred))

The accuracy for KNN Model Classification is: 
0.831754914621897


In [47]:
#predicting probablities
knn_prob = knn_model.predict_proba(X_test_vec)
print("Probabilities of sentiments (Irrelevant, Positive, Neutral, Negative):")
for i, (instance, prob) in enumerate(zip(X_test[:5], knn_prob[:5])):  # Printing probabilities of first 5 instances
    print(f"Instance {i + 1}:")
    print(f"Text: {instance}")
    print(f"Irrelevant: {prob[3]}")
    print(f"Positive: {prob[0]}")
    print(f"Neutral: {prob[2]}")
    print(f"Negative: {prob[1]}")

Probabilities of sentiments (Irrelevant, Positive, Neutral, Negative):
Instance 1:
Text: A ban has been imposed on Battlefield 4 player POLYSTATION7 see details bf4db.com player ban 100...
Irrelevant: 0.0
Positive: 1.0
Neutral: 0.0
Negative: 0.0
Instance 2:
Text:  
Irrelevant: 0.2
Positive: 0.2
Neutral: 0.2
Negative: 0.4
Instance 3:
Text: Thanks. Correction emailed
Irrelevant: 0.4
Positive: 0.2
Neutral: 0.2
Negative: 0.2
Instance 4:
Text: I love my heart stone.
Irrelevant: 0.0
Positive: 0.0
Neutral: 1.0
Negative: 0.0
Instance 5:
Text:  GhostRecon the games have their problems but I've definitely enjoyed my time keeping up the good work. pic.twitter.com rUTLeRbg4P
Irrelevant: 0.0
Positive: 0.2
Neutral: 0.6
Negative: 0.2
